In [1]:
import pandas as pd
import json
from geopy.distance import geodesic
import plotly.express as px
import plotly.graph_objects as go

In [2]:
BASE_PATH = '../almrrc2021/almrrc2021-data-training/model_build_inputs/'

In [3]:
# DRONE CONSTANTS
DRONE_MAX_SPEED = 7  # m/s

In [4]:
# Loading all the package data

package_data = json.load(open(BASE_PATH + 'package_data.json'))

flattened_package_data = []
for route_id, stops in package_data.items():
    for stop_id, packages in stops.items():
        for package_id, details in packages.items():
            record = {
                "route_id": route_id,
                "stop_id": stop_id,
                "package_id": package_id,
                "scan_status": details["scan_status"] if "scan_status" in details else None, # note model_apply_inputs does not have scan_status, model_build_inputs does
                "start_time_utc": details["time_window"]["start_time_utc"],
                "end_time_utc": details["time_window"]["end_time_utc"],
                "planned_service_time_seconds": details["planned_service_time_seconds"],
                "depth_cm": details["dimensions"]["depth_cm"],
                "height_cm": details["dimensions"]["height_cm"],
                "width_cm": details["dimensions"]["width_cm"],
            }
            flattened_package_data.append(record)

package_data_df = pd.DataFrame(flattened_package_data)

package_data_df.head()

,route_id,stop_id,package_id,scan_status,start_time_utc,end_time_utc,planned_service_time_seconds,depth_cm,height_cm,width_cm
0,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,AD,PackageID_9d7fdd03-f2cf-4c6f-9128-028258fc09ea,DELIVERED,NaN,NaN,59.3,25.4,7.6,17.8
1,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,AD,PackageID_5541e679-b7bd-4992-b288-e862f6c84ae7,DELIVERED,2018-07-27 16:00:00,2018-07-28 00:00:00,59.3,25.4,12.7,17.8
2,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,AD,PackageID_84d0295b-1adb-4a33-a65e-f7d6247c7a07,DELIVERED,NaN,NaN,59.3,39.4,7.6,31.8
3,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,AF,PackageID_15c6a204-ec5f-4ced-9c3d-472316cc7759,DELIVERED,2018-07-27 16:00:00,2018-07-28 00:00:00,27.0,30.0,3.0,27.4
4,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,AG,PackageID_3b28f781-242e-416e-9575-84c7188b8208,DELIVERED,NaN,NaN,45.0,25.4,12.7,17.8


In [5]:
# Loading all the route data

route_data = json.load(open(BASE_PATH + 'route_data.json'))

flattened_route_data = []

for route_id, info in route_data.items():
    for stop_id, stop_details in info['stops'].items():
        flattened_route_data.append({
            "route_id": route_id,
            "station_code": info['station_code'],
            "date": info['date_YYYY_MM_DD'],
            "departure_time_utc": info['departure_time_utc'],
            "executor_capacity_cm3": info['executor_capacity_cm3'],
            "route_score": info['route_score'],
            "stop_id": stop_id,
            "lat": stop_details['lat'],
            "lng": stop_details['lng'],
            "type": stop_details['type'],
            "zone_id": stop_details['zone_id']
        })

route_data_df = pd.DataFrame(flattened_route_data)

route_data_df.head()

,route_id,station_code,date,departure_time_utc,executor_capacity_cm3,route_score,stop_id,lat,lng,type,zone_id
0,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,AD,34.099611,-118.283062,Dropoff,P-12.3C
1,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,AF,34.101587,-118.291125,Dropoff,A-1.2D
2,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,AG,34.089727,-118.285530,Dropoff,A-2.1A
3,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,BA,34.096132,-118.292869,Dropoff,A-1.2C
4,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,BE,34.098482,-118.286243,Dropoff,P-13.3B


In [6]:
vehicle_travel_times = json.load(open(BASE_PATH + 'travel_times.json'))

In [7]:
def get_data_for_route(route_id):
    package_data = package_data_df[package_data_df['route_id'] == route_id]
    route_data = route_data_df[route_data_df['route_id'] == route_id]
    travel_times_route = vehicle_travel_times[route_id]

    return package_data.copy(), route_data.copy(), travel_times_route.copy()

In [8]:
# Given route data, it will return a dictionary
def calculate_drone_travel_time(route_data):
  drone_travel_time = {}

  for _, base_row in route_data.iterrows():
    drone_travel_time[base_row['stop_id']] = {}

    # calculate travel time to every other stop in the route
    for _, other_row in route_data.iterrows():
      if base_row['stop_id'] != other_row['stop_id']:
        base_location = (base_row['lat'], base_row['lng'])
        other_location = (other_row['lat'], other_row['lng'])

        distance = geodesic(base_location, other_location).meters

        drone_travel_time[base_row['stop_id']][other_row['stop_id']] = distance / DRONE_MAX_SPEED
        # drone_travel_time[base_row['stop_id']][other_row['stop_id']] += 10 # 5 second rise, 5 second drop

      else:
        drone_travel_time[base_row['stop_id']][other_row['stop_id']] = 0

  return drone_travel_time

In [9]:
# Instead of getting a random route we will get a route for which we have the proposed sequence

with open('proposed_sequences.json') as file:
    proposed_sequences_json = json.load(file)

proposed_route_ids = list(proposed_sequences_json.keys())

In [10]:
proposed_route_ids

['RouteID_15baae2d-bf07-4967-956a-173d4036613f',
 'RouteID_3f166f0e-fd2e-47ab-96a0-6cbc99cc6eef',
 'RouteID_5486294a-503f-4346-b8a9-862e988cbe7c',
 'RouteID_693060a6-88bb-4324-9e9c-925d5240263c',
 'RouteID_7f5d87f0-c39f-434f-bf3f-b159ef321909',
 'RouteID_9475872b-287f-4c2c-8e29-887766a4e090',
 'RouteID_a8f0009d-e50a-49c9-84d3-f9885ad14a54',
 'RouteID_bcc07fea-86d2-41e4-9a58-cfc78956dcc7',
 'RouteID_d1a8c3dd-fa67-455c-a68d-af2fd6aa5d91',
 'RouteID_e6687a05-2453-4edc-b86c-7558ab6d93f6',
 'RouteID_2b8df66d-fcd4-438e-931c-3b84b36a5c6b',
 'RouteID_f3261fad-5f97-44f6-ae7f-cf169f5d6452',
 'RouteID_fffd257c-3041-4736-be7a-5efea8af1173']

In [ ]:
# TODO:
# 1. Given a route
#     - Calculate the vehicle only travel time
#     - Create the route events json
#     - Create the route events path
# 2. Given a route, augment with drone